# Fixing Problems with the Golden Rule

In [1]:
import pandas as pd
from dfply import *

## The Golden Rule of Tabular Data

The **Golden Rule of Tabular Data** states that tables should be arranged with

* One variable per column
* One individual per row

Computer scientists call this [database normalization](https://en.wikipedia.org/wiki/Database_normalization); [Wickham (2014)](https://vita.had.co.nz/papers/tidy-data.pdf) calls this *tidy data*.

## Violations of the golden rule

Wickim identifies the following violations

* Column headers are values, not variable names.
* Multiple variables are stored in one column.
* Variables are stored in both rows and columns.
* Multiple types of observational units are stored in the same table.
* A single observational unit is stored in multiple tables.


## Example 1 - PEW Income Research
    
**Task:** Load the file `PEW_income_religion.csv`.  Identify the violation of the golden rule.


In [2]:
income = pd.read_csv('./data/PEW_income_religion.csv')
income.head(3)

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58


**Problem:** There are measurments in the the table heading

## Measurements in column labels?  Stack!

We can fix issues with informative column labels by stacking the data with `gather`

In [3]:
(income >>
  gather("income_range", 
         "count", 
         columns_from('<$10k')) >>
  head())

,religion,income_range,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don't know/refused,<$10k,15


## The `column_from` selection helper

* `dfply` selection helper.  
* Returns an `Intention`
* Expression for **all columns to the right** of the given column

Other helper functions described [here](https://github.com/kieferk/dfply#selection-filter-functions) and [Lecture 4.1.1](4_1_1_advanced_applications_of_dfply_select.ipynb).

In [4]:
columns_from('<$10k')

In [5]:
columns_from('<$10k').evaluate(income)

['<$10k', '$10-20k', '$20-30k', '$30-40k', '$40-50k', '$50-75k']

##  Example 2
    
Let's look at a dataset based on another example provided by Wickham.

In [6]:
weather = pd.read_csv('https://raw.githubusercontent.com/nickhould/tidy-data-python/master/data/weather-raw.csv')
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN


## Data in the column labels

`d1`, `d2`, etc. are days

In [7]:
weather_stacked = (weather >> 
                   gather('day',
                          'measurement',
                         columns_from('d1')))
weather_stacked >> head

,id,year,month,element,day,measurement
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


## Still violates the rule!

Note that 
* There are a lot of meaningless columns with no actual mmeasurements
* the `element` $\rightarrow$ *variable names*.
* Should be separate columns

In [9]:
weather_stacked >> head

,id,year,month,element,day,measurement
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


## Solution: Unstack the column containing variable names

In [10]:
weather_fixed = (weather_stacked 
                 >> filter_by(~X.measurement.isna()) 
                 >> spread(X.element, X.measurement))
weather_fixed

,id,year,month,day,tmax,tmin
0,MX17004,2010,2,d2,NaN,14.4
1,MX17004,2010,2,d3,NaN,14.4
2,MX17004,2010,3,d5,32.1,14.2
3,MX17004,2010,2,d2,27.3,NaN
4,MX17004,2010,2,d3,24.1,NaN


## Problems that require we reshape the data

The following problems 
* Measurement information in column labels
* Measurments on the same individual across multiple rows

## <font color="red"> Exercise 1 </font>
    
**Task:** Load the `rochester_mins_max_temp_2018.csv` data, contains data weather data for Rochester, MN. that is available at the [DNR website](https://www.dnr.state.mn.us/climate/historical/lcd.html?loc=rst). Note that `SM` and `AV` stand for *sum* and *average*, respectively.

1. Identify that problem with the current format.
2. Use `gather` and `spread` to fix the issue.

In [13]:
temps = pd.read_csv("./data/Rochester_temps_2019.csv")
temps

,ELEMENT,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,MEAN DAILY MAXIMUM,19.7,20.3,33.9,54.6,63.5,76.0,81.7,76.7,74.7,52.1,34.6,29.2
1,HIGHEST DAILY MAXIMUM,44.0,41.0,61.0,83.0,86.0,87.0,90.0,85.0,89.0,71.0,47.0,42.0
2,MEAN DAILY MINIMUM,4.4,2.4,17.9,35.1,44.3,56.5,62.6,56.7,55.5,35.6,21.5,13.9
3,LOWEST DAILY MINIMUM,-30.0,-15.0,-17.0,24.0,33.0,43.0,52.0,48.0,46.0,19.0,0.0,-8.0
4,AVERAGE DRY BULB,12.1,11.3,25.9,44.8,53.9,66.3,72.2,66.7,65.1,43.9,28.1,21.5
5,MEAN WET BULB,11.3,NaN,NaN,NaN,48.2,59.9,67.3,61.1,60.6,NaN,26.4,21.6
6,MEAN DEW POINT,8.3,NaN,NaN,NaN,43.0,55.2,64.3,57.8,57.5,NaN,22.6,18.3
7,NUM DAYS W MAXIMUM >= 90°,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,NUM DAYS W MAXIMUM <= 32°,25.0,24.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,17.0
9,NUM DAYS W MINIMUM <= 32°,31.0,27.0,27.0,11.0,0.0,0.0,0.0,0.0,0.0,14.0,29.0,31.0


The months are in columns and the element columns should be the column names.

In [18]:
temps_gathered = (temps
 >> gather('Month','Temps',columns_from('JAN')))
temps_gathered

,ELEMENT,Month,Temps
0,MEAN DAILY MAXIMUM,JAN,19.7
1,HIGHEST DAILY MAXIMUM,JAN,44.0
2,MEAN DAILY MINIMUM,JAN,4.4
3,LOWEST DAILY MINIMUM,JAN,-30.0
4,AVERAGE DRY BULB,JAN,12.1
...,...,...,...
127,MEAN DEW POINT,DEC,18.3
128,NUM DAYS W MAXIMUM >= 90°,DEC,0.0
129,NUM DAYS W MAXIMUM <= 32°,DEC,17.0
130,NUM DAYS W MINIMUM <= 32°,DEC,31.0


In [20]:
(temps_gathered
 >> spread(X.ELEMENT,X.Temps)
)

,Month,NUM DAYS W MINIMUM <= 0°,AVERAGE DRY BULB,HIGHEST DAILY MAXIMUM,LOWEST DAILY MINIMUM,MEAN DAILY MAXIMUM,MEAN DAILY MINIMUM,MEAN DEW POINT,MEAN WET BULB,NUM DAYS W MAXIMUM <= 32°,NUM DAYS W MINIMUM <= 32°,NUM DAYS W MAXIMUM >= 90°
0,APR,0.0,44.8,83.0,24.0,54.6,35.1,NaN,NaN,0.0,11.0,0.0
1,AUG,0.0,66.7,85.0,48.0,76.7,56.7,57.8,61.1,0.0,0.0,0.0
2,DEC,6.0,21.5,42.0,-8.0,29.2,13.9,18.3,21.6,17.0,31.0,0.0
3,FEB,15.0,11.3,41.0,-15.0,20.3,2.4,NaN,NaN,24.0,27.0,0.0
4,JAN,11.0,12.1,44.0,-30.0,19.7,4.4,8.3,11.3,25.0,31.0,0.0
5,JUL,0.0,72.2,90.0,52.0,81.7,62.6,64.3,67.3,0.0,0.0,1.0
6,JUN,0.0,66.3,87.0,43.0,76.0,56.5,55.2,59.9,0.0,0.0,0.0
7,MAR,6.0,25.9,61.0,-17.0,33.9,17.9,NaN,NaN,11.0,27.0,0.0
8,MAY,0.0,53.9,86.0,33.0,63.5,44.3,43.0,48.2,0.0,0.0,0.0
9,NOV,1.0,28.1,47.0,0.0,34.6,21.5,22.6,26.4,12.0,29.0,0.0
